In [2]:
# import necessary libraries
import numpy as np
import pandas as pd

In [3]:
def neg_zero(n):
    if(n < 0):
        p = 0
    else:
        p = n
    return(p)

In [4]:
# Use these values to calculate scores
gap_penalty = -2
match_award = 1
mismatch_penalty = -1

In [5]:
def smith_waterman_local_alignment(s1,s2):
    m = len(s1)
    n = len(s2)
    ga = np.zeros((n+1,m+1))
    w2 = ['-'] + list(s2)
    w1 = ['/'] + list(s1)
    for i in range(1,len(w1)):
        ga[0,i] = ga[0,i-1]-2
        ga[0,i] = neg_zero(ga[0,i])
    for j in range(1,len(w2)):
        ga[j,0] = ga[j-1,0]-2
        ga[j,0] = neg_zero(ga[j,0])
    for i in range(1,len(w1)):
        for j in range(1,len(w2)):
            if(w1[i]==w2[j]):
                ga[j,i] = max(neg_zero(ga[j-1,i] + gap_penalty),neg_zero(ga[j,i-1] + gap_penalty),neg_zero(ga[j-1,i-1] + match_award))
            elif(w1[i]=='-' or w2[j]=='-'):
                ga[j,i] = max(neg_zero(ga[j-1,i] + gap_penalty),neg_zero(ga[j,i-1] + gap_penalty),neg_zero(ga[j-1,i-1] + gap_penalty))
            else:
                ga[j,i] = max(neg_zero(ga[j-1,i] + gap_penalty),neg_zero(ga[j,i-1] + gap_penalty),neg_zero(ga[j-1,i-1] + mismatch_penalty))

            
            
    return(ga)   

In [6]:
s1 = "CCATACGA"
s2 = "CAGCTAGCG"

g = smith_waterman_local_alignment(s1,s2)
w2 = ['-'] + list(s2)
w1 = ['/'] + list(s1)
ga = pd.DataFrame(g, index = w2,columns = w1)
ga

,/,C,C,A,T,A,C,G,A
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
A,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0
G,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
C,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
T,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
A,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0
G,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
C,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [7]:
def match_score(x,y):
    if(x==y):
        return(match_award)
    elif(x=='-' or y=='-'):
        return(gap_penalty)
    else:
        return(mismatch_penalty)
        

In [8]:
def traceback(seq1,seq2):
    
    # find score using smith waterman alignment
    score = smith_waterman_local_alignment(seq1,seq2)
    
    # Create variables to store alignment
    align1 = ""
    align2 = ""

    result = np.where(score == np.amax(score))

    listOfCordinates = list(zip(result[0], result[1]))

    # strat from the maximum value from bottom
    i = listOfCordinates[-1][0]
    j = listOfCordinates[-1][1]


    # We'll use i and j to keep track of where we are in the matrix, just like above
    while i > 0 and j > 0: 
        score_current = score[i][j]
        score_diagonal = score[i-1][j-1]
        score_up = score[i][j-1]
        score_left = score[i-1][j]

        # Check to figure out which cell has the max score and
        # then update i and j to correspond to that cell.
        if score_current > 0:

            if (max(score_diagonal,score_up,score_left) == score_diagonal):
                align1 += seq1[j-1]
                align2 += seq2[i-1]
                i -= 1
                j -= 1
            elif (max(score_diagonal,score_up,score_left) == score_up):
                align1 += seq1[j-1]
                align2 += '-'
                j -= 1
            elif (max(score_diagonal,score_up,score_left) == score_left):
                align1 += '-'
                align2 += seq2[i-1]
                i -= 1
        else:
            break

    # Since we traversed the score matrix from the bottom right, our two sequences will be reversed.
    # These two lines reverse the order of the characters in each sequence.
    align1 = align1[::-1]
    align2 = align2[::-1]

    return(align1, align2)

In [10]:
s11 = "ttgact"
s12 = "ttgatc"
s21="gttcac"
s22="attcgt"
s31="ctcaggt"
s32="ctgagct"

s = smith_waterman_local_alignment(s11,s12)
output1, output2 = traceback(s21, s22)

print(output1 + "\n" + output2)    

ttc
ttc
